In [1]:
using TensorDecomposition, LinearAlgebra, Combinatorics, InvertedIndices, Symbolics, HomotopyContinuation

In [41]:
function rlim(n, d)
    if iseven(d)
         if n == 2
             return binomial(Int(n+d/2), n) - n
         else
             return binomial(Int(n+d/2), n) - n - 1
         end
     else 
         return binomial(Int(n+(d-1)/2), n)
     end
 end;

function cofactor(A::AbstractMatrix)
    ax = axes(A)
    out = similar(A, eltype(A), ax)
    for col in ax[1]
        for row in ax[2]
            out[col, row] = (-1)^(col + row) * det(A[Not(col), Not(row)])
        end
    end
    return out
end;

function alpha_iterator(::Val{N}, s, t=()) where {N}
    N <= 1 && return ((s, t...),) # Iterator over a single Tuple
    Iterators.flatten(alpha_iterator(Val(N-1), s-i, (i, t...)) for i in 0:s)
end

function monomialOrder(a, b)
    if a[1] > b[1]
        return true
    elseif a[1] < b[1]
        return false
    else 
        if maximum(a[2:end]) < maximum(b[2:end])
            return true
        elseif maximum(a[2:end]) > maximum(b[2:end])
            return false
        else
            return !isless(a[2:end], b[2:end])
        end 
    end
end

function convertIndices(x)
    d = sum(x)
    y = zeros(Int, d)
    c = 1
    for i=1:length(x)
        pow = x[i]
        y[c:c+pow-1] .= i
        c += pow
    end 
    return y .- 1
end;

function e(j, n)
    e = zeros(n)
    e[j] = 1
    return e
end;

function multBasis(basis, j, n)
    ej = e(j, n)
    ej[1] = -1
    return [Tuple(Int(y) for y in [x for x in b] + ej) for b in basis]
end

multMonomial(b, ej) = Tuple(Int(y) for y in [x for x in b] + ej);

function khatri_rao(A, d)
    if d == 1
        B = A
    else
        n, r = size(A)
        B = zeros(eltype(A), (n^d, r))
        for i=1:r 
            B[:, i] = kron(ntuple(x->A[:, i], d)...)
        end;
    end;
    return B
end

function krDrop(A, d)
    n = size(A)[1]-1
    inds = collect(with_replacement_combinations(1:n+1, d))
    inds = map(x->from_multiindex(x, n+1), inds)
    return khatri_rao(A, d)[inds, :]
end;

function from_multiindex(x, n)
    d = length(x)
    c = 0
    for i=1:d-1
        c += (x[i]-1)*n^(d-i)
    end
    return c + x[d]
end;

### Real tensor

In [155]:
n = 2
d = 4;
rlim(n, d)

4

In [156]:
r = 4

4

In [157]:
T, A_, L = randomRankedTensor(n+1, d, r, real=true);

In [158]:
Thank = hankMat(T);

In [170]:
monomials = reverse(sort(collect(alpha_iterator(Val(n+1), Int(d)))));
D = Dict()
for (i, mon) in enumerate(monomials)
   D[mon] = i 
end

basis = monomials[1:r]

4-element Vector{Tuple{Int64, Int64, Int64}}:
 (4, 0, 0)
 (3, 1, 0)
 (3, 0, 1)
 (2, 2, 0)

In [171]:
first_r = [D[b] for b in basis];

In [172]:
H0 = Float64.(Thank[first_r, first_r]);

In [173]:
H0_adj = cofactor(H0);
H0_det = det(H0);
H0_inv = inv(H0);

In [174]:
Hs = []
for i=2:n+1
    col_inds = [D[b] for b in multBasis(basis, i, n+1)]
    push!(Hs, Thank[first_r, col_inds])
end

In [175]:
vars = unique(reduce(vcat, ([reduce(vcat, HomotopyContinuation.variables.(H)) for H in Hs])))

2-element Vector{HomotopyContinuation.ModelKit.Variable}:
 h₁₆
 h₁₇

In [176]:
linearEquations = []

for i=1:n
    for j=i+1:n
        append!(linearEquations, HomotopyContinuation.expand.(Hs[i][n+2:end, :]*H0_adj*Hs[j][:, 2:n+1]-Hs[j][n+2:end, :]*H0_adj*Hs[i][:, 2:n+1]))
    end
end

In [180]:
i = 1
j = 2

HomotopyContinuation.expand.(Hs[i][n+2:end, :]*H0_inv*Hs[j][:, 2:n+1]-Hs[j][n+2:end, :]*H0_inv*Hs[i][:, 2:n+1])

1×2 Matrix{Expression}:
 8.58045546961438 + 1.01457335882854*h₁₆ - 1.0*h₁₇  …  8.15293142710098 + 1.03324430419377*h₁₆ - 1.01457335882854*h₁₇

In [188]:
alpha_i = multBasis(basis, i+1, n+1)[n+2]
alpha_j = multBasis(basis, j+1, n+1)[n+2]

beta_i = multBasis(basis, i+1, n+1)[3]
beta_j = multBasis(basis, j+1, n+1)[3]

Gij = Thank[vcat(first_r, D[alpha_i]), vcat(first_r, D[beta_j])]
Gji = Thank[vcat(first_r, D[alpha_j]), vcat(first_r, D[beta_i])]

HomotopyContinuation.expand.((det(Gij)-det(Gji))/H0_det)

-8.15293142710102 - 1.03324430419377*h₁₆ + 1.01457335882855*h₁₇ - 2.52203543189207e-14*h₁₈

In [189]:
HomotopyContinuation.expand(det(Gij)/H0_det)

-22.7257582763335 - 1.03324430419377*h₁₆ + 0.999999999999999*h₁₈

In [190]:
HomotopyContinuation.expand(det(Gji)/H0_det)

-14.5728268492325 - 1.01457335882855*h₁₇ + 1.00000000000001*h₁₈

In [117]:
A = []
b = []
for eq in linearEquations
    coeffs = HomotopyContinuation.coeffs_as_dense_poly(eq, vars, 1)
    push!(A, coeffs[1:end-1])
    push!(b, -coeffs[end])
end
A = reduce(hcat,A)';
b = Float64.(b);
# A = map!(x -> isapprox(x, 0, atol=1e-10) ? 0 : x, A, A);
# A = round.(A, digits=10);
# A = unique(A, dims=1);

rank(A)

10

In [20]:
sol = A\b;

Ms = []
for H in Hs
    push!(Ms, HomotopyContinuation.evaluate.(H*H0_inv, vars => sol))
end;

In [21]:
sol

2-element Vector{Float64}:
 -3.553470824541194
  1.65308479147949

In [31]:
HomotopyContinuation.expand(det(Thank[[1, 2, 3, 4, 7], [1, 2, 3, 4, 6]]))

0.839944744677702 + 0.298378435057643*h₁₆ - 0.132060227683639*h₁₈

In [27]:
linearEquations

2-element Vector{Any}:
  0.0120946539960882 + 0.0648384127282177*h₁₆ + 0.13206022768367*h₁₇
 -0.953095669664768 - 0.298378435057604*h₁₆ - 0.0648384127282174*h₁₇

In [32]:
HomotopyContinuation.expand(det(Thank[[1, 2, 3, 4, 8], [1, 2, 3, 4, 5]]))

-0.113150924987064 - 0.0648384127282355*h₁₇ - 0.132060227683638*h₁₈

In [38]:
P = permutedims((Complex.(L) .^ (1/4))).* A_;

In [56]:
P2 = krDrop(P, 2);

In [60]:
det(Thank[[1, 2, 3, 4], [1, 2, 3, 6]])

-0.298378435057596

In [59]:
det(P2[[1, 2, 3, 5], :])

0.0 + 0.1784211580067688im

In [71]:
A*[[0, 1];;[-1, 0]]/det(P2[1:4, :])

2×2 Matrix{ComplexF64}:
  0.0+0.363401im  -0.0-0.178421im
 -0.0-0.178421im   0.0+0.821072im

In [72]:
det(P2[[1, 2, 3, 4], :])

0.0 - 0.3634009186610109im

In [73]:
det(P2[[1, 2, 3, 5], :])

0.0 + 0.1784211580067688im

In [74]:
det(P2[[1, 2, 3, 6], :])

0.0 - 0.8210723191262199im

In [96]:
P_123 = det(P2[1:3, [1, 2, 3]])
P_124 = det(P2[1:3, [1, 2, 4]])
P_134 = det(P2[1:3, [1, 3, 4]])
P_234 = det(P2[1:3, [2, 3, 4]]);

In [97]:
-P[2, 1]^2*P_234 + P[2, 2]^2*P_134 - P[2, 3]^2*P_124 + P[2, 4]^2*P_123

0.0 - 0.363400918661011im

In [101]:
-P[3, 1]^2*P_234 + P[3, 2]^2*P_134 - P[3, 3]^2*P_124 + P[3, 4]^2*P_123

0.0 - 0.8210723191262198im

In [102]:
det(P2[[1, 2, 3, 6], :])

0.0 - 0.8210723191262199im

In [99]:
det(P2[[1, 2, 3, 5], :])

0.0 + 0.1784211580067688im